### 전국 장례식장 현황 PDF 추출

In [ ]:
import pdfplumber
import re
import os
from typing import List, Dict, Tuple, Set

# 전화번호 패턴
phone_pattern = re.compile(r'\d{2,3}-\d{3,4}-\d{4}')

In [ ]:
# 불필요한 공백 제거하고 빈 셀이면 아무것도 안하는 함수
def clean_cell(cell):
    return str(cell).strip() if cell is not None else ""

In [ ]:
# 한 셀에 다중항목을 포함할 경우 처리 함수
def post_process_record(record: Tuple[str, str, str, str]) -> List[Dict[str, str]]:
    sigungu, name_cell, address_cell, phone_cell = record
    
    names = name_cell.split('\n\n')
    addresses = address_cell.split('\n\n')
    phones = phone_cell.split('\n\n')
    
    # 가장 긴 리스트를 기준으로 레코드 수를 결정
    max_len = max(len(names), len(addresses), len(phones))
    
    # 리스트 길이가 다르면, 마지막 유효한 값으로 나머지를 채움
    if len(names) < max_len:
        names.extend([names[-1]] * (max_len - len(names)))
    if len(addresses) < max_len:
        addresses.extend([addresses[-1]] * (max_len - len(addresses)))
    if len(phones) < max_len:
        phones.extend([phones[-1]] * (max_len - len(phones)))
        
    results = []
    for name, address, phone in zip(names, addresses, phones):
        # 셀 내부의 단순 줄바꿈(\n)은 공백으로 처리
        name_clean = " ".join(name.split('\n')).strip()
        address_clean = " ".join(address.split('\n')).strip()
        phone_clean = " ".join(phone.split('\n')).strip()

        # 소재지는 '시군구' 정보의 첫 줄만 사용
        location = sigungu.split('\n')[0].strip()
        
        if name_clean and address_clean and phone_clean:
            results.append({
                "소재지": location,
                "장례식장 이름": name_clean,
                "주소": address_clean,
                "전화번호": phone_clean
            })
    return results

In [ ]:
# PDF에서 추출하는 함수
def extract_funeral_info(pdf_path):
    all_records_list = []
    processed_records_set = set()

    with pdfplumber.open(pdf_path) as pdf:
        current_sigungu = "정보 없음"
        
        for page in pdf.pages:
            tables = page.extract_tables() 
            
            if not tables:
                continue

            for table in tables:
                for row in table:
                    if not row:
                        continue
                        
                    cleaned_row = [clean_cell(cell) for cell in row]

                    # 헤더(제목 행)는 건너뛰기
                    if "시설명" in cleaned_row or "전화번호" in cleaned_row:
                        continue
                    
                    # 소재지(시군구) 셀 병합 처리
                    # 첫 번째 열에 값이 있으면 'current_sigungu'를 업데이트
                    if cleaned_row[0]:
                        current_sigungu = cleaned_row[0]

                    # 마지막 셀의 전화번호를 기준으로 데이터 행을 식별
                    last_cell = cleaned_row[-1]
                    if not phone_pattern.search(last_cell):
                        continue # 전화번호가 없으면 데이터 행이 아니므로 건너뛰기
                        
                    phone_data = last_cell
                    name_data = ""
                    address_data = ""

                    # 4열/3열 두 가지 테이블 포맷을 처리
                    # 포맷 1: [..., 시설명, 주소, 전화번호] (4열 이상)
                    # -3(시설명), -2(주소) 셀에 모두 내용이 있는지 확인
                    if len(cleaned_row) >= 4 and cleaned_row[-3] and cleaned_row[-2]:
                        name_data = cleaned_row[-3]
                        address_data = cleaned_row[-2]
                        
                    # 포맷 2: [..., "주소\n시설명", 전화번호] (3열 이상)
                    # -2(주소+시설명) 셀에만 내용이 있는지 확인
                    elif len(cleaned_row) >= 3 and cleaned_row[-2]:
                        parts = cleaned_row[-2].split('\n')
                        if len(parts) > 1:
                            # 휴리스틱: 보통 시설명이 마지막 줄에 온다.
                            name_data = parts[-1].strip()
                            address_data = " ".join(parts[:-1]).strip()
                        else:
                            # 분리 실패 시, 시설명으로 간주
                            name_data = cleaned_row[-2]
                            address_data = "정보 없음"
                    
                    # 유효한 데이터를 찾은 경우
                    if name_data and phone_data:
                        # 셀 내 다중 항목 처리를 위해 post_process 함수 호출
                        raw_record = (current_sigungu, name_data, address_data, phone_data)
                        
                        processed_list = post_process_record(raw_record)
                        for record_dict in processed_list:
                            # (소재지, 시설명, 주소)를 기준으로 중복을 체크
                            record_tuple = (
                                record_dict["소재지"], 
                                record_dict["장례식장 이름"], 
                                record_dict["주소"] 
                            )
                            if record_tuple not in processed_records_set:
                                all_records_list.append(record_dict)
                                processed_records_set.add(record_tuple)
                                
    return all_records_list

In [ ]:
pdf_file_path = os.path.join('data', 'funeral', '전국_장례식장_현황.pdf')
abs_pdf_path = os.path.abspath(pdf_file_path)

extracted_data = extract_funeral_info(abs_pdf_path)

# 결과 출력
print(f"--- 총 {len(extracted_data)}개의 고유한 장례식장 정보를 추출했습니다. ---")

print("\n--- 추출 데이터 샘플 (앞 10개) ---")
for i, item in enumerate(extracted_data[:10]):
    print(f"[{i+1:03d}] 소재지: {item['소재지']} | 장례식장 이름: {item['장례식장 이름']} | 주소: {item['주소']} | 전화번호: {item['전화번호']}")
    
print("\n--- 추출 데이터 샘플 (뒤 10개) ---")
for i, item in enumerate(extracted_data[-10:]):
    print(f"[{len(extracted_data)-9+i:03d}] 소재지: {item['소재지']} | 장례식장 이름: {item['장례식장 이름']} | 주소: {item['주소']} | 전화번호: {item['전화번호']}")

--- 총 1087개의 고유한 장례식장 정보를 추출했습니다. ---

--- 추출 데이터 샘플 (앞 10개) ---
[001] 소재지: 서울 | 장례식장 이름: 서울적십자병원장례식장 | 주소: 서울특별시 종로구 새문안로 9 (평동, 적십자병원) | 전화번호: 02-2002-8444
[002] 소재지: 서울 | 장례식장 이름: 서울대학교병원장례식장 | 주소: 서울특별시 종로구 대학로 101, 서울대학교병원 (연건동) | 전화번호: 02-2072-2020
[003] 소재지: 서울 | 장례식장 이름: 국립중앙의료원 장례식장 | 주소: 서울특별시 중구 을지로 245 (을지로6가, 국립중앙의료원) | 전화번호: 02-2262-4800
[004] 소재지: 서울 | 장례식장 이름: 순천향서울장례식장 | 주소: 서울특별시 용산구 대사관로 59 (한남동, 순천향대학교병원) | 전화번호: 02-797-4444
[005] 소재지: 서울 | 장례식장 이름: 한양대학병원장례식장 | 주소: 서울특별시 성동구 왕십리로 222-1 (사근동, 한양대학부속병원) | 전화번호: 02-2290-9442
[006] 소재지: 서울 | 장례식장 이름: 건국대학교병원장례식장 | 주소: 서울특별시 광진구 능동로 120-1 (화양동) | 전화번호: 02-2030-7900
[007] 소재지: 서울 | 장례식장 이름: 혜민병원장례식장 | 주소: 서울특별시 광진구 자양로 85 (자양동) | 전화번호: 02-444-1552
[008] 소재지: 서울 | 장례식장 이름: 경희의료원장례식장 | 주소: 서울특별시 동대문구 경희대로 23, 경희의료원장례식장 (회기동, 경희의료원) | 전화번호: 02-958-9721
[009] 소재지: 서울 | 장례식장 이름: 삼육서울병원 추모관 | 주소: 서울특별시 동대문구 망우로 82 (휘경동) | 전화번호: 02-2215-4444
[010] 소재지: 서울 | 장례식장 이름: 서울특별시 동부병원 장례식장 | 주소: 서울특별시 동대문구 무학로 124 (용두동, 시립동부병원) | 전화번호:

In [6]:
extracted_data[0]

{'소재지': '서울',
 '장례식장 이름': '서울적십자병원장례식장',
 '주소': '서울특별시 종로구 새문안로 9 (평동, 적십자병원)',
 '전화번호': '02-2002-8444'}

In [12]:
import pandas as pd

df = pd.DataFrame(extracted_data[:], index=range(len(extracted_data)))
df

,소재지,장례식장 이름,주소,전화번호
0,서울,서울적십자병원장례식장,"서울특별시 종로구 새문안로 9 (평동, 적십자병원)",02-2002-8444
1,서울,서울대학교병원장례식장,"서울특별시 종로구 대학로 101, 서울대학교병원 (연건동)",02-2072-2020
2,서울,국립중앙의료원 장례식장,"서울특별시 중구 을지로 245 (을지로6가, 국립중앙의료원)",02-2262-4800
3,서울,순천향서울장례식장,"서울특별시 용산구 대사관로 59 (한남동, 순천향대학교병원)",02-797-4444
4,서울,한양대학병원장례식장,"서울특별시 성동구 왕십리로 222-1 (사근동, 한양대학부속병원)",02-2290-9442
...,...,...,...,...
1082,경남,거창적십자병원장례식장,"경상남도 거창군 거창읍 중앙로 91 (상림리, 거창적십자병원)",055-944-4482
1083,경남,거창서경병원장례식장( 거창한국병원장례식장),"경상남도 거창군 거창읍 송정1길 24-13 (송정리, 서경병원)",055-945-0130
1084,경남,합천추모공원장례식장,경상남도 합천군 합천읍 합천호수로 1613 (합천리),055-934-4444
1085,경남,합천고려병원 장례식장,경상남도 합천군 대양면 대야로 737-20 (정양리),055-931-4464


In [13]:
df.to_csv('./data/funeral/funeral_homes.csv', index=False)